In [1]:
##Data ingestion
from langchain_community.document_loaders import TextLoader
loader=TextLoader("speech.txt")
text_documents=loader.load()
text_documents

[Document(page_content='I am happy to join with you today in what will go down in history as the greatest demonstration for freedom in the history of our nation.\n\nFive score years ago, a great American, in whose symbolic shadow we stand today, signed the Emancipation Proclamation. This momentous decree came as a great beacon of hope to millions of slaves, who had been seared in the flames of whithering injustice. It came as a joyous daybreak to end the long night of their captivity. But one hundred years later, the colored America is still not free. One hundred years later, the life of the colored American is still sadly crippled by the manacle of segregation and the chains of discrimination.\n\nOne hundred years later, the colored American lives on a lonely island of poverty in the midst of a vast ocean of material prosperity. One hundred years later, the colored American is still languishing in the corners of American society and finds himself an exile in his own land So we have co

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
##os.environ['OPENAI_API_KEY']='sk-proj-fhqAcvXIWo11Cl1PQhBBT3BlbkFJA7E9OUVR36ZPlLqUdJLHgh'

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
#load,chunk and index the contect of the html page

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post_content","post_header")
                     )),)

text_documents=loader.load()
text_documents

[Document(page_content='\n      LLM Powered Autonomous Agents\n    ', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'})]

In [4]:
## pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('Schwanseestr.pdf')
docs=loader.load()
docs

[Document(page_content='Objekt : Schwanseestrasse  \n81549 München - Giesing   \nEtage: 3.stock  \nErbaut: 1965  \nFläche: 60qm \n 2 Zimmer , Küche, Bad mit Wanne, Balkon, \nLoggia, Flur, Abstellkammer + Kellerabteil  \n Bodenbelag : Parkett ( Wohnzi ) + Laminat \n(Schlafzi ) + Rest Fliesen \n Bad wurde 2020 neu gefliest, Küche im \n2016 eingebaut, neu gefliest  \n    ', metadata={'source': 'Schwanseestr.pdf', 'page': 0}),
 Document(page_content='Aktuelle kosten:  \nWohngeld  ca.400Eur/Monat  \nLetzte Jahresabrechnung  2023: Umlagefähig 1099,62 + Heizg /WW 1297,75  + \nGrundsteuer 229,76 eur \n \nAktuelle Mieteinahmen: 1000 Miete + 100 BK + 130 HK \nVermietet bis 31.8.2024  \n    \n', metadata={'source': 'Schwanseestr.pdf', 'page': 1}),
 Document(page_content='Fotos :  \nEingang            Flur  ', metadata={'source': 'Schwanseestr.pdf', 'page': 2}),
 Document(page_content='Küche  ', metadata={'source': 'Schwanseestr.pdf', 'page': 3}),
 Document(page_content='Wohnzimmer  \n', metadata=

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
documents=text_splitter.split_documents(docs)
documents[:5]


[Document(page_content='Objekt : Schwanseestrasse  \n81549 München - Giesing   \nEtage: 3.stock  \nErbaut: 1965  \nFläche: 60qm \n 2 Zimmer , Küche, Bad mit Wanne, Balkon, \nLoggia, Flur, Abstellkammer + Kellerabteil  \n Bodenbelag : Parkett ( Wohnzi ) + Laminat \n(Schlafzi ) + Rest Fliesen \n Bad wurde 2020 neu gefliest, Küche im \n2016 eingebaut, neu gefliest', metadata={'source': 'Schwanseestr.pdf', 'page': 0}),
 Document(page_content='Aktuelle kosten:  \nWohngeld  ca.400Eur/Monat  \nLetzte Jahresabrechnung  2023: Umlagefähig 1099,62 + Heizg /WW 1297,75  + \nGrundsteuer 229,76 eur \n \nAktuelle Mieteinahmen: 1000 Miete + 100 BK + 130 HK \nVermietet bis 31.8.2024', metadata={'source': 'Schwanseestr.pdf', 'page': 1}),
 Document(page_content='Fotos :  \nEingang            Flur', metadata={'source': 'Schwanseestr.pdf', 'page': 2}),
 Document(page_content='Küche', metadata={'source': 'Schwanseestr.pdf', 'page': 3}),
 Document(page_content='Wohnzimmer', metadata={'source': 'Schwanseestr.p

In [6]:
##Vector Embedding and Vector Store
from langchain_community.embeddings import OpenAIEmbeddings
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db=Chroma.from_documents(documents[:20],OpenAIEmbeddings())

c:\Users\kasib\Desktop\LanchainSeries\Chatbot\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
db

In [8]:
##vector db
query="Where is the location of the apartment?"
result=db.similarity_search(query)
result[3].page_content

'Objekt : Schwanseestrasse  \n81549 München - Giesing   \nEtage: 3.stock  \nErbaut: 1965  \nFläche: 60qm \n 2 Zimmer , Küche, Bad mit Wanne, Balkon, \nLoggia, Flur, Abstellkammer + Kellerabteil  \n Bodenbelag : Parkett ( Wohnzi ) + Laminat \n(Schlafzi ) + Rest Fliesen \n Bad wurde 2020 neu gefliest, Küche im \n2016 eingebaut, neu gefliest'

In [9]:
from langchain_community.llms import Ollama
##Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm


Ollama()

In [10]:
##Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template("""
                                        Answer the following question based only on the provided context.
                                        Think step by step before providing a detailed answer.
                                        I will tip you $2345 if the user finds the answer helpful.
                                        <context>
                                        {context}
                                        <context>
                                        Questions:{input}""")
                               

In [11]:
##Chain intorduction
##Create stuff document chain

from  langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)

Retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [12]:
retriever=db.as_retriever()
retriever


VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000024300FB9FC0>)

In [13]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [18]:
retrieval_chain.invoke({"input":"Translate whole document to english"})



{'input': 'Translate whole document to english',
 'context': [Document(page_content='Fotos :  \nEingang            Flur', metadata={'page': 2, 'source': 'Schwanseestr.pdf'}),
  Document(page_content='Wohnzimmer', metadata={'page': 4, 'source': 'Schwanseestr.pdf'}),
  Document(page_content='Küche', metadata={'page': 3, 'source': 'Schwanseestr.pdf'}),
  Document(page_content='Schlafzimmer', metadata={'page': 5, 'source': 'Schwanseestr.pdf'})],
 'answer': 'Thank you for providing the context! Based on the given text, I can translate it to English as follows:\n\nFotos :  \nEingang - Entrance\nFlur - Floor\nWohnzimmer - Living Room\nKüche - Kitchen\nSchlafzimmer - Bedroom\n\nSo, the translation of the entire document to English is:\n\nFotos :  \nEntrance - Floor\nLiving Room - Kitchen\nKitchen - Bedroom'}

In [15]:
##FAISS Vector Database
from langchain_community.vectorstores import FAISS
db1=FAISS.from_documents(documents[:20],OpenAIEmbeddings())

In [17]:
##vector db croma
query="Where is the location of the apartment?"
result=db.similarity_search(query)
result[0].page_content

[Document(page_content='Fotos :  \nEingang            Flur', metadata={'page': 2, 'source': 'Schwanseestr.pdf'}),
 Document(page_content='Wohnzimmer', metadata={'page': 4, 'source': 'Schwanseestr.pdf'}),
 Document(page_content='Küche', metadata={'page': 3, 'source': 'Schwanseestr.pdf'}),
 Document(page_content='Aktuelle kosten:  \nWohngeld  ca.400Eur/Monat  \nLetzte Jahresabrechnung  2023: Umlagefähig 1099,62 + Heizg /WW 1297,75  + \nGrundsteuer 229,76 eur \n \nAktuelle Mieteinahmen: 1000 Miete + 100 BK + 130 HK \nVermietet bis 31.8.2024', metadata={'page': 1, 'source': 'Schwanseestr.pdf'})]